>## 2022313356 김지헌_머신러닝 기반 Windows 악성코드 자동탐지

### 전처리 과정

In [1]:
import pandas as pd

In [2]:
#사용불가
api = pd.read_csv('API_Functions.csv')
print(api.shape)

(29505, 21920)


##### dll imported

In [4]:
dll = pd.read_csv('DLLs_Imported.csv')
print(dll.shape)

(29498, 631)


In [5]:
dll.columns

Index(['SHA256', 'Type', 'advapi32.dll', 'kernel32.dll', 'vspmsg.dll',
       'ole32.dll', 'oleaut32.dll', 'psapi.dll', 'setupapi.dll', 'shlwapi.dll',
       ...
       'odbccp32.dll', 'api-ms-win-crt-environment-l1-1-0.dll',
       'api-ms-win-core-memory-l1-1-3.dll',
       'api-ms-win-core-datetime-l1-1-0.dll',
       'api-ms-win-core-psapi-ansi-l1-1-0.dll',
       'api-ms-win-core-fibers-l1-1-0.dll', 'api-ms-win-core-file-l2-1-0.dll',
       'api-ms-win-core-sysinfo-l1-2-0.dll', 'dbgeng.dll', 'd3d11.dll'],
      dtype='object', length=631)

In [11]:
#isNull
dll.columns[dll.isnull().any()]

Index([], dtype='object')

In [8]:
dll_sha = dll['SHA256']

In [9]:
#중복값이 3개 존재하나, SHA256 특성상 해쉬 충돌로 중복되었을 확률 존재
len(set(dll_sha)), len(dll_sha)

(29495, 29498)

##### PE header

In [2]:
pe_h = pd.read_csv('PE_Header.csv')
pe_h

,SHA256,Type,e_magic,e_cblp,e_cp,e_crlc,e_cparhdr,e_minalloc,e_maxalloc,e_ss,...,SizeOfImage,SizeOfHeaders,CheckSum,Subsystem,DllCharacteristics,SizeOfStackReserve,SizeOfHeapReserve,SizeOfHeapCommit,LoaderFlags,NumberOfRvaAndSizes
0,dacbe8cb72dd746539792a50e84965fefef73feaa07b5d...,0,23117,144,3,0,4,0,65535,0,...,139264,4096,0,2,1344,1048576,1048576,4096,0,16
1,d3dc7512ce75db33b2c3063fa99245e9ca9fe3b086462f...,0,23117,144,3,0,4,0,65535,0,...,49152,512,0,2,34112,1048576,1048576,4096,0,16
2,b350fac81533f02981dc2176ed17163177d92d9405758e...,0,23117,144,3,0,4,0,65535,0,...,532480,512,0,2,34144,1048576,1048576,4096,0,16
3,dfee618043a47b7b09305df0ca460559d9f567ee246c7b...,0,23117,144,3,0,4,0,65535,0,...,1368064,4096,1366781,2,1024,1048576,1048576,4096,0,16
4,c7b2e4e4fb2fcc44c953673ff57c3d14bdf5d2008f35e9...,0,23117,144,3,0,4,0,65535,0,...,32768,512,64362,2,1344,1048576,1048576,4096,0,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29802,d1c923ec32d09be96adf3ba85e1855402f782b74437534...,6,23117,144,3,0,4,0,65535,0,...,974848,512,0,2,34112,1048576,1048576,4096,0,16
29803,d1dfc3203d99c0d54351bebd876762f27c0073e3cfef9a...,6,23117,144,3,0,4,0,65535,0,...,434176,512,0,2,34112,1048576,1048576,4096,0,16
29804,d1fea52507fc97ff419f8dd2ea8ecf689fb7c066cf8f18...,6,23117,144,3,0,4,0,65535,0,...,958464,512,0,2,34112,1048576,1048576,4096,0,16
29805,d201d8488d316c9ab7d537545efeda284568a652f23db1...,6,23117,144,3,0,4,0,65535,0,...,1114112,512,0,2,34112,1048576,1048576,4096,0,16


In [10]:
#isNull
pe_h.columns[pe_h.isnull().any()]

Index([], dtype='object')

In [29]:
pe_h_sha = pe_h['SHA256']
len(set(pe_h_sha)), len(pe_h_sha)

(29807, 29807)

In [15]:
#type와의 Corr 계산
pe_h_2 = pe_h.iloc[:,1:]
corr_with_type = pe_h_2.corr()['Type'].drop('Type')

In [16]:
sorted(corr_with_type)

[nan,
 -0.45644647244887804,
 -0.40265680520639635,
 -0.35872567907251923,
 -0.20150265239801202,
 -0.20150265239799875,
 -0.20150265239787996,
 -0.17150391557039066,
 -0.12465160406887137,
 -0.11862943006694175,
 -0.10724004128211238,
 -0.09678201724137882,
 -0.09194946157520445,
 -0.08826191065698062,
 -0.07702481727760568,
 -0.06538485328031558,
 -0.04884602474396896,
 -0.048432621986309654,
 -0.039323775727838024,
 -0.035143646936126434,
 -0.028962416913253287,
 -0.021337787905860106,
 -0.018128804196517957,
 -0.017524976747652073,
 -0.01361182643712018,
 -0.007714676029361793,
 -0.0073742856218206995,
 -0.0018107402280263007,
 -0.0015911229753635887,
 -0.0013297517660546784,
 0.009299284214017198,
 0.010012302773256395,
 0.010265067670847871,
 0.027849628576923524,
 0.03377641370746158,
 0.03502551170389729,
 0.03815175634774968,
 0.03944005271908247,
 0.041776851997691475,
 0.04373753928400159,
 0.04527460951096478,
 0.046043756812611435,
 0.04670150428705597,
 0.0469704290423177

##### PE Section

In [12]:
pe_s = pd.read_csv('PE_Section.csv')
pe_s

,SHA256,Type,text_Misc_VirtualSize,text_VirtualAddress,text_SizeOfRawData,text_PointerToRawData,text_PointerToRelocations,text_PointerToLinenumbers,text_NumberOfRelocations,text_NumberOfLinenumbers,...,tls_Characteristics,pdata_Misc_VirtualSize,pdata_VirtualAddress,pdata_SizeOfRawData,pdata_PointerToRawData,pdata_PointerToRelocations,pdata_PointerToLinenumbers,pdata_NumberOfRelocations,pdata_NumberOfLinenumbers,pdata_Characteristics
0,dacbe8cb72dd746539792a50e84965fefef73feaa07b5d...,0,114580,8192,114688,4096,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,d3dc7512ce75db33b2c3063fa99245e9ca9fe3b086462f...,0,16436,8192,16896,512,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,b350fac81533f02981dc2176ed17163177d92d9405758e...,0,506420,8192,506880,512,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,dfee618043a47b7b09305df0ca460559d9f567ee246c7b...,0,1312036,8192,1314816,4096,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,c7b2e4e4fb2fcc44c953673ff57c3d14bdf5d2008f35e9...,0,2660,8192,3072,512,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29755,d1c923ec32d09be96adf3ba85e1855402f782b74437534...,6,893524,8192,893952,512,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29756,d1dfc3203d99c0d54351bebd876762f27c0073e3cfef9a...,6,386780,8192,387072,512,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29757,d1fea52507fc97ff419f8dd2ea8ecf689fb7c066cf8f18...,6,923524,8192,923648,512,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29758,d201d8488d316c9ab7d537545efeda284568a652f23db1...,6,1087404,8192,1087488,512,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
#isNull
pe_s.columns[pe_s.isnull().any()]

Index([], dtype='object')

In [28]:
pe_s_sha = pe_s['SHA256']
len(set(pe_s_sha)), len(pe_s_sha)

(29760, 29760)

In [19]:
pe_s_2 = pe_s.iloc[:,1:]
corr_with_type = pe_s_2.corr()['Type'].drop('Type')
sorted(corr_with_type)

[-0.546037925538818,
 -0.25299485833856217,
 -0.14659504921293398,
 -0.12882517085876719,
 -0.1285724791612325,
 -0.12120978525037336,
 -0.11481782054869319,
 -0.09224469098915042,
 -0.07284719400120597,
 -0.048887436229703686,
 -0.040524300121911355,
 -0.03982729937383095,
 -0.028778248612731268,
 -0.02795358155965259,
 -0.024384791997202858,
 -0.022913646062847216,
 -0.017538247920868073,
 -0.017093346146586943,
 -0.016910271272670013,
 -0.016696878863646205,
 -0.016668906570286142,
 -0.01646600558216448,
 -0.01598513684288004,
 -0.015721041695766606,
 -0.015312375964002251,
 -0.01496637247848507,
 -0.01405165452035587,
 -0.013517581878608175,
 -0.01336423701542762,
 -0.012965533374154849,
 -0.011181188769052653,
 -0.01038435727025489,
 -0.010199500974898688,
 -0.010170294990489048,
 -0.00926699799940579,
 -0.007188855585113857,
 0.013914940129317287,
 0.21916113508511076,
 0.21972533801333097,
 nan,
 nan,
 nan,
 nan,
 -0.1863027531813249,
 -0.10116442929389323,
 -0.09952488408889712

3개의 파일 간 공유하지 않는 File 확인

In [27]:
s11 = set(dll['SHA256'])
s22 = set(pe_h['SHA256'])
s33 = set(pe_s['SHA256'])

print(len(s11-s22), (s11-s22))
print("="*50)
print(len(s22-s11), (s22-s11))
print("="*50)

print(len(s11-s33), (s11-s33))
print("="*50)
print(len(s33-s11), (s33-s11))
print("="*50)

print(len(s22-s33), (s22-s33))
print("="*50)
print(len(s33-s22), (s33-s22))
print("="*50)



1 {'afcb3cc4c9ae4900e0ae6d6363638e92a816c46e12e830e0f11e34e71cb14651'}
313 {'...'}
3 {'0f0c23ea76170d902f7924227c8af2d423e58e6130f706e477382cc4cc411ebe', 'afcb3cc4c9ae4900e0ae6d6363638e92a816c46e12e830e0f11e34e71cb14651', '0f2c192b767541bb6d3539e3be7ab8264e746a757cd9a9c308f705f3140f13a9'}
268 {'42d570cd852e4bc909f7b72cdaf45b43a43f32b07364bda976e2b00ee9b385a7', '78749911faeac0032bb0c8562761fa6ee3fb85f37e099f5169d9dcc29c8024bd', 'a271681a6f3d7c1a898cea80333917003e57494ac1fb9c331dd042f7483d2ebf', '532fbe194acb281ef6be139e0724a46d9936606b190b62bc57ba0859b2b61b8c', '9847005465ae681d1d9533697106492027a1d55b64b0ca1b6f2a79730a5140a4', '975e410cd3ed5a1bd7bf3aa10c41f61e2bdf8781048722b592ba6139f0bde7fc', 'a79b2ea4e8db2dd2e6729d73534d1a03083e17d7a2d598cfed22c07076f0679b', '68408b8c3a9aa56febfe6cfbeb75aec02ce5c12e5d67a9f8854939899771d2f8', '9b93483aa508bf9211b53d4f89e11c5a87f0fef6b6ef5c7ab2a17089ccac0a2a', 'b1dab76c6446187ba124c1b4e5552c2ac9f4cc8b68e09ca42820a9eadbad06db', '5bef8b9b04fd97ace67ce223